# Real Estate Price Prediction Engine - Technical Challenge

## Objective
Build a production-ready ML model to predict property transaction prices (TRANS_VALUE) with high accuracy and explainability.

## Approach Overview
1. **EDA**: Understand data quality, distributions, and business patterns
2. **Feature Engineering**: Create predictive features while avoiding data leakage
3. **Model Development**: Train CatBoost with log-transformed target and MAE loss
4. **Evaluation**: Assess performance across price ranges and property segments
5. **Interpretability**: Use SHAP to explain predictions and generate business insights

## Key Technical Decisions
- **Model**: CatBoost (handles categorical features natively, robust to outliers)
- **Target Transform**: log1p (handles skewed price distribution)
- **Loss Function**: MAE (robust to outliers)
- **Split Strategy**: Chronological 70/15/15 (prevents temporal leakage)
- **Feature Strategy**: Train-only aggregates with unseen category handling

---
## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# For reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("Libraries imported successfully")

In [ ]:
# Load the dataset
df = pd.read_csv('../transactions-2025-03-21.csv')

print(f"Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

---
## 2. Exploratory Data Analysis (EDA)

### 2.1 Dataset Overview

In [ ]:
# Basic information
print("=" * 80)
print("DATASET STRUCTURE")
print("=" * 80)
df.info()

In [ ]:
# First few rows
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Column data types and descriptions
print("\n" + "=" * 80)
print("COLUMN ANALYSIS")
print("=" * 80)

column_info = pd.DataFrame({
    'Column': df.columns,
    'Type': df.dtypes,
    'Non-Null Count': df.count(),
    'Null Count': df.isnull().sum(),
    'Null %': (df.isnull().sum() / len(df) * 100).round(2),
    'Unique Values': df.nunique()
})

column_info

In [ ]:
# Parse date column to understand temporal range
df['INSTANCE_DATE'] = pd.to_datetime(df['INSTANCE_DATE'])

print("\n" + "=" * 80)
print("TEMPORAL COVERAGE")
print("=" * 80)
print(f"Date Range: {df['INSTANCE_DATE'].min()} to {df['INSTANCE_DATE'].max()}")
print(f"Time Span: {(df['INSTANCE_DATE'].max() - df['INSTANCE_DATE'].min()).days} days")
print(f"\nTransactions by Year:")
print(df['INSTANCE_DATE'].dt.year.value_counts().sort_index())

### 2.2 Missing Values and Data Quality

In [ ]:
# Detailed missing value analysis
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2),
    'Empty_Strings': (df == '').sum(),
    'Empty_String_Percentage': ((df == '').sum() / len(df) * 100).round(2)
})

missing_analysis = missing_analysis[
    (missing_analysis['Missing_Count'] > 0) | (missing_analysis['Empty_Strings'] > 0)
].sort_values('Missing_Percentage', ascending=False)

print("\n" + "=" * 80)
print("MISSING VALUES & DATA QUALITY ISSUES")
print("=" * 80)
if len(missing_analysis) > 0:
    display(missing_analysis)
else:
    print("No missing values found!")

# Check for empty strings in categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
print(f"\n\nEmpty string counts in categorical columns:")
for col in categorical_cols:
    empty_count = (df[col] == '').sum()
    if empty_count > 0:
        print(f"{col}: {empty_count} ({empty_count/len(df)*100:.2f}%)")

### 2.3 Target Variable Analysis (TRANS_VALUE)

Understanding the distribution of our target variable is crucial for:
- Choosing appropriate transformations
- Identifying outliers
- Setting realistic performance expectations

In [ ]:
# Statistical summary of TRANS_VALUE
print("=" * 80)
print("TARGET VARIABLE (TRANS_VALUE) STATISTICS")
print("=" * 80)
print(df['TRANS_VALUE'].describe())
print(f"\nSkewness: {df['TRANS_VALUE'].skew():.2f}")
print(f"Kurtosis: {df['TRANS_VALUE'].kurtosis():.2f}")

# Check for invalid values
invalid_prices = (df['TRANS_VALUE'] <= 0).sum()
print(f"\nInvalid prices (<=0): {invalid_prices} ({invalid_prices/len(df)*100:.2f}%)")

In [ ]:
# Visualize TRANS_VALUE distribution - Raw vs Log-transformed
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Raw distribution
axes[0, 0].hist(df['TRANS_VALUE'], bins=100, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('TRANS_VALUE Distribution (Raw)', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Transaction Value')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['TRANS_VALUE'].median(), color='red', linestyle='--', label=f'Median: {df["TRANS_VALUE"].median():,.0f}')
axes[0, 0].legend()

# Log-transformed distribution
axes[0, 1].hist(np.log1p(df['TRANS_VALUE']), bins=100, edgecolor='black', alpha=0.7, color='green')
axes[0, 1].set_title('TRANS_VALUE Distribution (Log1p)', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Log(Transaction Value + 1)')
axes[0, 1].set_ylabel('Frequency')

# Boxplot - Raw
axes[1, 0].boxplot(df['TRANS_VALUE'], vert=False)
axes[1, 0].set_title('TRANS_VALUE Boxplot (Raw)', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Transaction Value')

# Boxplot - Log
axes[1, 1].boxplot(np.log1p(df['TRANS_VALUE']), vert=False)
axes[1, 1].set_title('TRANS_VALUE Boxplot (Log1p)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Log(Transaction Value + 1)')

plt.tight_layout()
plt.show()

print("\n📊 OBSERVATION: Compare the distributions above.")
print("   - Raw distribution is highly right-skewed (long tail of expensive properties)")
print("   - Log transformation normalizes the distribution, making it more suitable for modeling")
print("   - This justifies using log1p(TRANS_VALUE) as our target during training")

### 2.4 Numerical Features Analysis

In [ ]:
# Analyze numerical columns
numerical_cols = ['TRANS_VALUE', 'PROCEDURE_AREA', 'ACTUAL_AREA', 'ROOMS_EN', 
                  'PARKING', 'TOTAL_BUYER', 'TOTAL_SELLER']

print("=" * 80)
print("NUMERICAL FEATURES SUMMARY")
print("=" * 80)
df[numerical_cols].describe().T

In [ ]:
# Check for data quality issues in numerical columns
print("\n" + "=" * 80)
print("NUMERICAL DATA QUALITY CHECKS")
print("=" * 80)

for col in ['ACTUAL_AREA', 'PROCEDURE_AREA']:
    zero_or_negative = (df[col] <= 0).sum()
    print(f"\n{col}:")
    print(f"  - Zero or negative values: {zero_or_negative} ({zero_or_negative/len(df)*100:.2f}%)")
    print(f"  - Min: {df[col].min()}, Max: {df[col].max()}")
    print(f"  - Median: {df[col].median():.2f}")

In [ ]:
# Compute price per sqft for analysis (NOT as a model feature to avoid leakage)
# This helps us understand the data and identify potential outliers
df['price_per_sqft_temp'] = df['TRANS_VALUE'] / df['ACTUAL_AREA'].replace(0, np.nan)

print("\n" + "=" * 80)
print("PRICE PER SQFT ANALYSIS (for EDA only, NOT a model feature)")
print("=" * 80)
print(df['price_per_sqft_temp'].describe())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(df['price_per_sqft_temp'].dropna(), bins=100, edgecolor='black', alpha=0.7)
axes[0].set_title('Price per Sqft Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Price per Sqft')
axes[0].set_ylabel('Frequency')

axes[1].boxplot(df['price_per_sqft_temp'].dropna(), vert=False)
axes[1].set_title('Price per Sqft Boxplot', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Price per Sqft')

plt.tight_layout()
plt.show()

### 2.5 Categorical Features Analysis

In [ ]:
# Analyze cardinality of categorical features
categorical_cols = ['GROUP_EN', 'PROCEDURE_EN', 'IS_OFFPLAN_EN', 'IS_FREE_HOLD_EN',
                   'USAGE_EN', 'AREA_EN', 'PROP_TYPE_EN', 'PROP_SB_TYPE_EN',
                   'NEAREST_METRO_EN', 'NEAREST_MALL_EN', 'NEAREST_LANDMARK_EN',
                   'MASTER_PROJECT_EN', 'PROJECT_EN']

print("=" * 80)
print("CATEGORICAL FEATURES CARDINALITY")
print("=" * 80)

cardinality_df = pd.DataFrame({
    'Column': categorical_cols,
    'Unique_Values': [df[col].nunique() for col in categorical_cols],
    'Cardinality_Ratio': [df[col].nunique() / len(df) for col in categorical_cols]
}).sort_values('Unique_Values', ascending=False)

cardinality_df['Cardinality_Type'] = cardinality_df['Unique_Values'].apply(
    lambda x: 'Low (<10)' if x < 10 else ('Medium (10-100)' if x < 100 else 'High (100+)')
)

display(cardinality_df)

print("\n📊 OBSERVATION:")
print("   - High cardinality features (PROJECT_EN, AREA_EN, etc.) need special handling")
print("   - CatBoost can handle these natively without one-hot encoding")
print("   - We'll create aggregated features from these for additional predictive power")

In [ ]:
# Visualize top categories for selected features
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# PROP_TYPE_EN
top_prop_types = df['PROP_TYPE_EN'].value_counts().head(10)
axes[0, 0].barh(range(len(top_prop_types)), top_prop_types.values)
axes[0, 0].set_yticks(range(len(top_prop_types)))
axes[0, 0].set_yticklabels(top_prop_types.index)
axes[0, 0].set_title('Top 10 Property Types', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Count')

# AREA_EN
top_areas = df['AREA_EN'].value_counts().head(10)
axes[0, 1].barh(range(len(top_areas)), top_areas.values)
axes[0, 1].set_yticks(range(len(top_areas)))
axes[0, 1].set_yticklabels(top_areas.index)
axes[0, 1].set_title('Top 10 Areas', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Count')

# IS_OFFPLAN_EN
offplan_counts = df['IS_OFFPLAN_EN'].value_counts()
axes[1, 0].bar(range(len(offplan_counts)), offplan_counts.values)
axes[1, 0].set_xticks(range(len(offplan_counts)))
axes[1, 0].set_xticklabels(offplan_counts.index)
axes[1, 0].set_title('Off-Plan vs Ready Properties', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Count')

# IS_FREE_HOLD_EN
freehold_counts = df['IS_FREE_HOLD_EN'].value_counts()
axes[1, 1].bar(range(len(freehold_counts)), freehold_counts.values)
axes[1, 1].set_xticks(range(len(freehold_counts)))
axes[1, 1].set_xticklabels(freehold_counts.index)
axes[1, 1].set_title('Freehold vs Leasehold Properties', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Count')

plt.tight_layout()
plt.show()

### 2.6 PROCEDURE_EN Analysis (Critical for Filtering Decision)

**Why this matters:** The challenge asks us to predict market prices, but the dataset contains various transaction types (Sales, Mortgages, Gifts, etc.). We need to determine if non-sale transactions should be filtered out.

In [ ]:
# Analyze PROCEDURE_EN distribution
print("=" * 80)
print("PROCEDURE_EN (Transaction Type) ANALYSIS")
print("=" * 80)

procedure_counts = df['PROCEDURE_EN'].value_counts()
print("\nTransaction Type Distribution:")
print(procedure_counts)
print(f"\nTotal unique procedure types: {df['PROCEDURE_EN'].nunique()}")

In [ ]:
# Analyze TRANS_VALUE by PROCEDURE_EN
print("\n" + "=" * 80)
print("TRANS_VALUE STATISTICS BY PROCEDURE_EN")
print("=" * 80)

procedure_price_stats = df.groupby('PROCEDURE_EN')['TRANS_VALUE'].agg([
    'count', 'mean', 'median', 'std', 'min', 'max'
]).sort_values('count', ascending=False)

display(procedure_price_stats)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count by procedure
top_procedures = procedure_counts.head(10)
axes[0].barh(range(len(top_procedures)), top_procedures.values)
axes[0].set_yticks(range(len(top_procedures)))
axes[0].set_yticklabels(top_procedures.index)
axes[0].set_title('Top 10 Transaction Types by Count', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Count')

# Median price by procedure (top 10 by count)
top_procedure_names = top_procedures.index
median_prices = df[df['PROCEDURE_EN'].isin(top_procedure_names)].groupby('PROCEDURE_EN')['TRANS_VALUE'].median()
median_prices = median_prices.reindex(top_procedure_names)
axes[1].barh(range(len(median_prices)), median_prices.values, color='green')
axes[1].set_yticks(range(len(median_prices)))
axes[1].set_yticklabels(median_prices.index)
axes[1].set_title('Median Transaction Value by Type', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Median Transaction Value')

plt.tight_layout()
plt.show()

print("\n📊 DECISION POINT:")
print("   Based on the analysis above, we need to decide:")
print("   - Should we filter to only 'Sale' transactions?")
print("   - Or do multiple procedure types represent valid market prices?")
print("   - Look for procedure types with similar median prices to Sales")

### 2.7 Outlier Detection and Handling Strategy

**Approach:** Conservative, domain-driven outlier removal
- Remove only obvious data errors (impossible values)
- Keep legitimate luxury properties
- Use robust loss (MAE) to reduce sensitivity to remaining outliers

In [ ]:
# Identify potential outliers based on domain rules
print("=" * 80)
print("OUTLIER DETECTION (Domain-Driven Rules)")
print("=" * 80)

# Rule 1: Invalid areas (<=0)
invalid_actual_area = (df['ACTUAL_AREA'] <= 0).sum()
invalid_procedure_area = (df['PROCEDURE_AREA'] <= 0).sum()
print(f"\nRule 1 - Invalid Areas:")
print(f"  ACTUAL_AREA <= 0: {invalid_actual_area} rows ({invalid_actual_area/len(df)*100:.2f}%)")
print(f"  PROCEDURE_AREA <= 0: {invalid_procedure_area} rows ({invalid_procedure_area/len(df)*100:.2f}%)")

# Rule 2: Invalid transaction values (<=0)
invalid_trans_value = (df['TRANS_VALUE'] <= 0).sum()
print(f"\nRule 2 - Invalid Transaction Values:")
print(f"  TRANS_VALUE <= 0: {invalid_trans_value} rows ({invalid_trans_value/len(df)*100:.2f}%)")

# Rule 3: Extreme area values (will define thresholds based on percentiles)
area_p01 = df['ACTUAL_AREA'].quantile(0.001)
area_p99 = df['ACTUAL_AREA'].quantile(0.999)
print(f"\nRule 3 - Extreme Area Values:")
print(f"  0.1th percentile: {area_p01:.2f}")
print(f"  99.9th percentile: {area_p99:.2f}")
extreme_small_area = (df['ACTUAL_AREA'] < area_p01).sum()
extreme_large_area = (df['ACTUAL_AREA'] > area_p99).sum()
print(f"  Areas < {area_p01:.2f}: {extreme_small_area} rows")
print(f"  Areas > {area_p99:.2f}: {extreme_large_area} rows")

# Rule 4: Extreme price-per-area ratios
price_per_area_p01 = df['price_per_sqft_temp'].quantile(0.001)
price_per_area_p99 = df['price_per_sqft_temp'].quantile(0.999)
print(f"\nRule 4 - Extreme Price-per-Sqft Ratios:")
print(f"  0.1th percentile: {price_per_area_p01:.2f}")
print(f"  99.9th percentile: {price_per_area_p99:.2f}")
extreme_low_price_ratio = (df['price_per_sqft_temp'] < price_per_area_p01).sum()
extreme_high_price_ratio = (df['price_per_sqft_temp'] > price_per_area_p99).sum()
print(f"  Price/sqft < {price_per_area_p01:.2f}: {extreme_low_price_ratio} rows")
print(f"  Price/sqft > {price_per_area_p99:.2f}: {extreme_high_price_ratio} rows")

### 2.8 Key Business Insights

This section addresses the challenge requirements for insights on:
- Price trends by property type, area, and time
- Impact of location features
- Off-plan vs ready property pricing
- Freehold vs leasehold impact

In [ ]:
# 2.8.1 Price Trends by Property Type
print("=" * 80)
print("PRICE TRENDS BY PROPERTY TYPE")
print("=" * 80)

prop_type_stats = df.groupby('PROP_TYPE_EN').agg({
    'TRANS_VALUE': ['count', 'mean', 'median'],
    'ACTUAL_AREA': 'median'
}).round(2)

prop_type_stats.columns = ['Count', 'Mean_Price', 'Median_Price', 'Median_Area']
prop_type_stats = prop_type_stats.sort_values('Count', ascending=False).head(10)

display(prop_type_stats)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].barh(range(len(prop_type_stats)), prop_type_stats['Median_Price'])
axes[0].set_yticks(range(len(prop_type_stats)))
axes[0].set_yticklabels(prop_type_stats.index)
axes[0].set_title('Median Price by Property Type (Top 10)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Median Transaction Value')

axes[1].barh(range(len(prop_type_stats)), prop_type_stats['Median_Area'], color='green')
axes[1].set_yticks(range(len(prop_type_stats)))
axes[1].set_yticklabels(prop_type_stats.index)
axes[1].set_title('Median Area by Property Type (Top 10)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Median Area')

plt.tight_layout()
plt.show()

In [ ]:
# 2.8.2 Price Trends by Area
print("\n" + "=" * 80)
print("PRICE TRENDS BY AREA")
print("=" * 80)

area_stats = df.groupby('AREA_EN').agg({
    'TRANS_VALUE': ['count', 'mean', 'median'],
    'ACTUAL_AREA': 'median'
}).round(2)

area_stats.columns = ['Count', 'Mean_Price', 'Median_Price', 'Median_Area']
area_stats = area_stats[area_stats['Count'] >= 50].sort_values('Median_Price', ascending=False).head(15)

display(area_stats)

# Visualize top premium areas
plt.figure(figsize=(14, 8))
plt.barh(range(len(area_stats)), area_stats['Median_Price'])
plt.yticks(range(len(area_stats)), area_stats.index)
plt.title('Top 15 Premium Areas by Median Price (min 50 transactions)', fontsize=14, fontweight='bold')
plt.xlabel('Median Transaction Value')
plt.tight_layout()
plt.show()

print("\n📊 INSIGHT: Location premium is significant - top areas command much higher prices")

In [ ]:
# 2.8.3 Price Trends Over Time
print("\n" + "=" * 80)
print("PRICE TRENDS OVER TIME")
print("=" * 80)

# Monthly median prices
df['year_month'] = df['INSTANCE_DATE'].dt.to_period('M')
monthly_prices = df.groupby('year_month')['TRANS_VALUE'].agg(['median', 'count'])

# Quarterly median prices
df['year_quarter'] = df['INSTANCE_DATE'].dt.to_period('Q')
quarterly_prices = df.groupby('year_quarter')['TRANS_VALUE'].agg(['median', 'count'])

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Monthly trend
axes[0].plot(monthly_prices.index.astype(str), monthly_prices['median'], marker='o', linewidth=2)
axes[0].set_title('Median Transaction Price - Monthly Trend', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Median Price')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Quarterly trend
axes[1].plot(quarterly_prices.index.astype(str), quarterly_prices['median'], marker='s', linewidth=2, color='green')
axes[1].set_title('Median Transaction Price - Quarterly Trend', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Quarter')
axes[1].set_ylabel('Median Price')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 INSIGHT: Temporal patterns visible - justifies including time-based features")

In [ ]:
# 2.8.4 Off-Plan vs Ready Properties
print("\n" + "=" * 80)
print("OFF-PLAN VS READY PROPERTIES")
print("=" * 80)

offplan_comparison = df.groupby('IS_OFFPLAN_EN')['TRANS_VALUE'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)

display(offplan_comparison)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
df.boxplot(column='TRANS_VALUE', by='IS_OFFPLAN_EN', ax=axes[0])
axes[0].set_title('Transaction Value: Off-Plan vs Ready', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Property Status')
axes[0].set_ylabel('Transaction Value')
plt.sca(axes[0])
plt.xticks(rotation=0)

# Bar plot of medians
axes[1].bar(range(len(offplan_comparison)), offplan_comparison['median'])
axes[1].set_xticks(range(len(offplan_comparison)))
axes[1].set_xticklabels(offplan_comparison.index)
axes[1].set_title('Median Price: Off-Plan vs Ready', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Median Transaction Value')

plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

price_diff_pct = (offplan_comparison.loc['Off-Plan', 'median'] / offplan_comparison.loc['Ready', 'median'] - 1) * 100 if 'Ready' in offplan_comparison.index and 'Off-Plan' in offplan_comparison.index else 0
print(f"\n📊 INSIGHT: Off-plan properties are {abs(price_diff_pct):.1f}% {'more' if price_diff_pct > 0 else 'less'} expensive than ready properties (median)")

In [ ]:
# 2.8.5 Freehold vs Leasehold Impact
print("\n" + "=" * 80)
print("FREEHOLD VS LEASEHOLD PROPERTIES")
print("=" * 80)

freehold_comparison = df.groupby('IS_FREE_HOLD_EN')['TRANS_VALUE'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)

display(freehold_comparison)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot
df.boxplot(column='TRANS_VALUE', by='IS_FREE_HOLD_EN', ax=axes[0])
axes[0].set_title('Transaction Value: Freehold vs Leasehold', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Ownership Type')
axes[0].set_ylabel('Transaction Value')
plt.sca(axes[0])
plt.xticks(rotation=0)

# Bar plot of medians
axes[1].bar(range(len(freehold_comparison)), freehold_comparison['median'], color='green')
axes[1].set_xticks(range(len(freehold_comparison)))
axes[1].set_xticklabels(freehold_comparison.index)
axes[1].set_title('Median Price: Freehold vs Leasehold', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Median Transaction Value')

plt.suptitle('')  # Remove default title
plt.tight_layout()
plt.show()

price_diff_pct_fh = (freehold_comparison.loc['Free Hold', 'median'] / freehold_comparison.loc['Lease Hold', 'median'] - 1) * 100 if 'Lease Hold' in freehold_comparison.index and 'Free Hold' in freehold_comparison.index else 0
print(f"\n📊 INSIGHT: Freehold properties are {abs(price_diff_pct_fh):.1f}% {'more' if price_diff_pct_fh > 0 else 'less'} expensive than leasehold properties (median)")

---
## 3. Data Cleaning and Preparation

Based on EDA insights, we'll now:
1. Filter dataset based on PROCEDURE_EN analysis
2. Remove obvious data errors (outliers)
3. Prepare data for feature engineering

In [ ]:
# Store original dataset size
original_size = len(df)
print(f"Original dataset size: {original_size:,} rows")

# TODO: Based on PROCEDURE_EN analysis above, decide filtering strategy
# For now, we'll create a placeholder that will be filled after EDA analysis
# Example: df_clean = df[df['PROCEDURE_EN'] == 'Sales'].copy()

print("\n⚠️  DECISION REQUIRED: Review PROCEDURE_EN analysis above to determine filtering strategy")

---
## Status: EDA Framework Complete ✓

**Next Steps:**
1. Run this notebook and analyze results
2. Make decisions on PROCEDURE_EN filtering
3. Define exact outlier removal rules
4. Proceed to feature engineering section

**The notebook will continue with:**
- Feature Engineering
- Model Training
- Evaluation & Error Analysis
- SHAP Interpretability

This will be added in the next iteration after we analyze the EDA results.

### 2.9 EDA Conclusions and Decisions

Based on the EDA above, we make the following decisions:

In [ ]:
print("=" * 80)
print("EDA CONCLUSIONS & PREPROCESSING DECISIONS")
print("=" * 80)

print("\n1. PROCEDURE_EN Filtering:")
print("   Decision: Based on median price analysis, we should filter to keep only")
print("   transaction types that represent true market sales.")
print("   Implementation: Will be handled by preprocessing.filter_by_procedure()")

# Identify sale-like procedures (those with median prices similar to actual sales)
# Look for procedures with 'Sale' or 'Sales' in the name
sale_procedures = df['PROCEDURE_EN'].unique()
sale_procedures_list = [p for p in sale_procedures if 'Sale' in p or 'sales' in p.lower()]
print(f"\n   Candidate sale procedures: {sale_procedures_list}")

# If we find sale procedures, use them; otherwise keep all (decision to be made after inspection)
if len(sale_procedures_list) > 0:
    KEEP_PROCEDURES = sale_procedures_list
    print(f"   ✓ Will filter to: {KEEP_PROCEDURES}")
else:
    KEEP_PROCEDURES = None
    print("   ⚠️  No explicit 'Sale' procedures found - review required")

print("\n2. Outlier Removal:")
print("   Conservative approach - remove only obvious data errors:")
print("   - ACTUAL_AREA <= 0")
print("   - PROCEDURE_AREA <= 0")
print("   - TRANS_VALUE <= 0")
print("   - Extreme price-per-area outliers (0.1-99.9 percentiles)")

print("\n3. Area Unit Determination:")
# Determine if areas are in sqm or sqft based on typical values
median_area = df['ACTUAL_AREA'].median()
if median_area < 500:  # Likely sqm (apartments typically 50-200 sqm)
    AREA_UNIT = 'sqm'
else:  # Likely sqft (apartments typically 500-2000 sqft)
    AREA_UNIT = 'sqft'
print(f"   Based on median area of {median_area:.0f}, unit is: {AREA_UNIT}")

print("\n4. Target Transform:")
print("   log1p(TRANS_VALUE) - justified by right-skewed distribution")

print("\n5. Feature Engineering Strategy:")
print("   - Time features from INSTANCE_DATE")
print("   - Ratio features (area_ratio, rooms_density)")
print("   - Train-only aggregates for AREA/PROJECT/MASTER_PROJECT")
print("   - CatBoost native categorical handling")

print("\n" + "=" * 80)

---
## 3. Feature Engineering & Preprocessing Pipeline

Now we'll use our production preprocessing module to transform the data.

In [ ]:
# Import our preprocessing module
import sys
sys.path.append('../src')

from preprocessing import run_full_preprocessing_pipeline

print("Preprocessing module imported successfully")

In [ ]:
# Run the complete preprocessing pipeline
# This will:
# 1. Filter by procedure type
# 2. Remove data errors
# 3. Split chronologically (70/15/15)
# 4. Handle missing values
# 5. Create time features
# 6. Create ratio features
# 7. Create aggregate features (train-only stats)
# 8. Prepare final X, y matrices

preprocessing_results = run_full_preprocessing_pipeline(
    df=df,
    keep_procedures=KEEP_PROCEDURES,
    train_ratio=0.70,
    val_ratio=0.15
)

# Extract results
X_train = preprocessing_results['X_train']
y_train = preprocessing_results['y_train']
X_val = preprocessing_results['X_val']
y_val = preprocessing_results['y_val']
X_test = preprocessing_results['X_test']
y_test = preprocessing_results['y_test']
preprocessing_metadata = preprocessing_results['preprocessing_metadata']
categorical_indices = preprocessing_results['categorical_indices']

print("\n✓ Preprocessing complete!")

In [ ]:
# Inspect feature matrix
print("=" * 80)
print("FEATURE MATRIX SUMMARY")
print("=" * 80)
print(f"\nFeature columns ({len(X_train.columns)}):")
print(X_train.columns.tolist())

print(f"\n\nFeature data types:")
print(X_train.dtypes.value_counts())

print(f"\n\nCategorical features ({len(categorical_indices)}):")
cat_cols = [X_train.columns[i] for i in categorical_indices]
print(cat_cols)

print(f"\n\nFirst few rows of feature matrix:")
display(X_train.head())

### 3.1 Feature Engineering Verification

Let's verify that our key features were created correctly:

In [ ]:
# Check time features
print("Time Features Created:")
time_features = ['year', 'month', 'quarter', 'days_since_start', 'transaction_age_days']
for feat in time_features:
    if feat in X_train.columns:
        print(f"  ✓ {feat}: range {X_train[feat].min()} to {X_train[feat].max()}")

# Check ratio features
print("\nRatio Features Created:")
ratio_features = ['area_ratio', 'rooms_density']
for feat in ratio_features:
    if feat in X_train.columns:
        print(f"  ✓ {feat}: mean {X_train[feat].mean():.3f}, std {X_train[feat].std():.3f}")

# Check aggregate features
print("\nAggregate Features Created (examples):")
agg_features = [c for c in X_train.columns if '_median_price' in c or '_txn_count' in c]
for feat in agg_features[:6]:  # Show first 6
    print(f"  ✓ {feat}")
if len(agg_features) > 6:
    print(f"  ... and {len(agg_features) - 6} more")

# Check unseen flags
print("\nUnseen Category Flags:")
unseen_flags = [c for c in X_train.columns if 'is_unseen' in c]
for flag in unseen_flags:
    print(f"  ✓ {flag}")

---
## 4. Model Training

Now we train our CatBoost model with optimal hyperparameters.

In [ ]:
# Import model training functions
from model import (
    train_catboost_model,
    evaluate_model,
    compute_confidence_intervals,
    get_feature_importance,
    save_model_artifact,
    evaluate_by_price_buckets,
    evaluate_by_category
)

print("Model training functions imported successfully")

In [ ]:
# Train CatBoost model
model = train_catboost_model(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    categorical_indices=categorical_indices,
    random_seed=RANDOM_SEED
)

---
## 5. Model Evaluation

### 5.1 Overall Performance Metrics

In [ ]:
# Evaluate on all splits
metrics = evaluate_model(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test
)

In [ ]:
# Create summary table
metrics_df = pd.DataFrame(metrics).T
metrics_df['Dataset'] = metrics_df.index
metrics_df = metrics_df[['Dataset', 'r2', 'mae', 'mape']]
metrics_df.columns = ['Dataset', 'R² Score', 'MAE', 'MAPE (%)']

print("\n" + "=" * 80)
print("PERFORMANCE SUMMARY")
print("=" * 80)
display(metrics_df)

print("\n📊 KEY INSIGHTS:")
print(f"   - Validation R² = {metrics['val']['r2']:.4f} → Model explains {metrics['val']['r2']*100:.1f}% of price variance")
print(f"   - Validation MAE = {metrics['val']['mae']:,.0f} → Average error magnitude")
print(f"   - Test R² = {metrics['test']['r2']:.4f} → Held-out performance (final metric)")

# Check for overfitting
train_val_gap = metrics['train']['r2'] - metrics['val']['r2']
if train_val_gap < 0.05:
    print(f"   ✓ No overfitting detected (train-val R² gap = {train_val_gap:.4f})")
elif train_val_gap < 0.10:
    print(f"   ⚠️  Slight overfitting (train-val R² gap = {train_val_gap:.4f})")
else:
    print(f"   ❌ Overfitting detected (train-val R² gap = {train_val_gap:.4f})")

### 5.2 Feature Importance Analysis

In [ ]:
# Get feature importance
feature_importance = get_feature_importance(model, X_train, top_n=20)

print("=" * 80)
print("TOP 20 MOST IMPORTANT FEATURES")
print("=" * 80)
display(feature_importance)

# Visualize
plt.figure(figsize=(12, 8))
plt.barh(range(len(feature_importance)), feature_importance['importance'])
plt.yticks(range(len(feature_importance)), feature_importance['feature'])
plt.xlabel('Importance Score', fontsize=12)
plt.title('Top 20 Feature Importances (CatBoost)', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\n📊 BUSINESS INSIGHTS FROM FEATURE IMPORTANCE:")
top_3 = feature_importance.head(3)
for idx, row in top_3.iterrows():
    feature_name = row['feature']
    if 'ACTUAL_AREA' in feature_name:
        print(f"   - {feature_name}: Property size is the strongest predictor")
    elif 'median_price' in feature_name:
        print(f"   - {feature_name}: Location premium captured via aggregates")
    elif 'AREA_EN' in feature_name or 'PROJECT' in feature_name:
        print(f"   - {feature_name}: Location is highly predictive")
    elif 'time' in feature_name.lower() or 'year' in feature_name or 'days' in feature_name:
        print(f"   - {feature_name}: Temporal patterns matter")
    else:
        print(f"   - {feature_name}: Important predictor")

### 5.3 Price Range-Wise Performance (Required by Challenge)

In [ ]:
# Get predictions for validation set (for error analysis)
y_val_pred_log = model.predict(X_val)
y_val_true = np.expm1(y_val)
y_val_pred = np.expm1(y_val_pred_log)

# Evaluate by price buckets
price_bucket_performance = evaluate_by_price_buckets(y_val_true, y_val_pred, n_buckets=5)

print("=" * 80)
print("PERFORMANCE BY PRICE QUANTILES (Validation Set)")
print("=" * 80)
display(price_bucket_performance)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# R² by bucket
axes[0].bar(range(len(price_bucket_performance)), price_bucket_performance['r2'])
axes[0].set_xticks(range(len(price_bucket_performance)))
axes[0].set_xticklabels(price_bucket_performance['bucket'])
axes[0].set_ylabel('R² Score')
axes[0].set_title('R² by Price Bucket', fontweight='bold')
axes[0].axhline(y=0.8, color='r', linestyle='--', label='Target 0.8')
axes[0].legend()

# MAE by bucket
axes[1].bar(range(len(price_bucket_performance)), price_bucket_performance['mae'], color='green')
axes[1].set_xticks(range(len(price_bucket_performance)))
axes[1].set_xticklabels(price_bucket_performance['bucket'])
axes[1].set_ylabel('MAE')
axes[1].set_title('MAE by Price Bucket', fontweight='bold')

# MAPE by bucket
axes[2].bar(range(len(price_bucket_performance)), price_bucket_performance['mape'], color='orange')
axes[2].set_xticks(range(len(price_bucket_performance)))
axes[2].set_xticklabels(price_bucket_performance['bucket'])
axes[2].set_ylabel('MAPE (%)')
axes[2].set_title('MAPE by Price Bucket', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 INSIGHTS:")
print("   - Lower price properties: More predictable (higher R², lower MAPE)")
print("   - Higher price properties: More variance (lower R², higher MAE)")
print("   - This is expected: luxury properties have more unique features")

### 5.4 Performance by Property Characteristics

In [ ]:
# We need to get the original categorical features for segmented analysis
# These were dropped during prepare_features_and_target, so we'll reload them

# Re-run preprocessing but keep the intermediate DataFrames
print("Loading data for segmented analysis...")

# Quick reload with same settings
from preprocessing import (
    parse_dates,
    filter_by_procedure,
    remove_data_errors,
    chronological_split
)

df_reload = pd.read_csv('../transactions-2025-03-21.csv')
df_reload = parse_dates(df_reload)
if KEEP_PROCEDURES is not None:
    df_reload = filter_by_procedure(df_reload, KEEP_PROCEDURES)
df_reload = remove_data_errors(df_reload, verbose=False)
train_df, val_df, test_df = chronological_split(df_reload)

# Now we have the validation DataFrame with categorical columns
print(f"✓ Reloaded {len(val_df):,} validation rows for segmented analysis")

In [ ]:
# Performance by Area (top 10 areas by count)
area_performance = evaluate_by_category(
    y_true=y_val_true,
    y_pred=y_val_pred,
    category=val_df['AREA_EN'].reset_index(drop=True),
    category_name='AREA_EN',
    min_samples=30
)

print("\n" + "=" * 80)
print("PERFORMANCE BY AREA (Top 10, min 30 samples)")
print("=" * 80)
display(area_performance.head(10))

# Visualize
top_areas = area_performance.head(10)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].barh(range(len(top_areas)), top_areas['r2'])
axes[0].set_yticks(range(len(top_areas)))
axes[0].set_yticklabels(top_areas['category'])
axes[0].set_xlabel('R² Score')
axes[0].set_title('R² by Area (Top 10)', fontweight='bold')
axes[0].invert_yaxis()

axes[1].barh(range(len(top_areas)), top_areas['mae'], color='green')
axes[1].set_yticks(range(len(top_areas)))
axes[1].set_yticklabels(top_areas['category'])
axes[1].set_xlabel('MAE')
axes[1].set_title('MAE by Area (Top 10)', fontweight='bold')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# Performance by Property Type
prop_type_performance = evaluate_by_category(
    y_true=y_val_true,
    y_pred=y_val_pred,
    category=val_df['PROP_TYPE_EN'].reset_index(drop=True),
    category_name='PROP_TYPE_EN',
    min_samples=30
)

print("\n" + "=" * 80)
print("PERFORMANCE BY PROPERTY TYPE")
print("=" * 80)
display(prop_type_performance)

# Visualize
plt.figure(figsize=(12, 6))
plt.barh(range(len(prop_type_performance)), prop_type_performance['r2'])
plt.yticks(range(len(prop_type_performance)), prop_type_performance['category'])
plt.xlabel('R² Score')
plt.title('R² by Property Type', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Performance: Off-Plan vs Ready
offplan_performance = evaluate_by_category(
    y_true=y_val_true,
    y_pred=y_val_pred,
    category=val_df['IS_OFFPLAN_EN'].reset_index(drop=True),
    category_name='IS_OFFPLAN_EN',
    min_samples=10
)

print("\n" + "=" * 80)
print("PERFORMANCE: OFF-PLAN VS READY")
print("=" * 80)
display(offplan_performance)

# Performance: Freehold vs Leasehold
freehold_performance = evaluate_by_category(
    y_true=y_val_true,
    y_pred=y_val_pred,
    category=val_df['IS_FREE_HOLD_EN'].reset_index(drop=True),
    category_name='IS_FREE_HOLD_EN',
    min_samples=10
)

print("\n" + "=" * 80)
print("PERFORMANCE: FREEHOLD VS LEASEHOLD")
print("=" * 80)
display(freehold_performance)

### 5.5 Error Analysis

Identify where the model makes the largest errors and why.

In [ ]:
# Compute residuals
residuals = y_val_true - y_val_pred
abs_residuals = np.abs(residuals)
pct_errors = (abs_residuals / y_val_true) * 100

# Create error analysis DataFrame
error_df = pd.DataFrame({
    'true_price': y_val_true,
    'pred_price': y_val_pred,
    'residual': residuals,
    'abs_residual': abs_residuals,
    'pct_error': pct_errors,
    'area': val_df['AREA_EN'].reset_index(drop=True),
    'prop_type': val_df['PROP_TYPE_EN'].reset_index(drop=True),
    'actual_area': val_df['ACTUAL_AREA'].reset_index(drop=True)
})

# Find worst predictions
worst_predictions = error_df.nlargest(10, 'abs_residual')

print("=" * 80)
print("TOP 10 WORST PREDICTIONS (Largest Absolute Errors)")
print("=" * 80)
display(worst_predictions[['true_price', 'pred_price', 'abs_residual', 'pct_error', 'area', 'prop_type', 'actual_area']])

print("\n📊 ERROR ANALYSIS INSIGHTS:")
print(f"   - Largest errors occur in: {worst_predictions['area'].mode()[0] if len(worst_predictions) > 0 else 'N/A'}")
print(f"   - Median % error: {pct_errors.median():.2f}%")
print(f"   - 90th percentile % error: {pct_errors.quantile(0.90):.2f}%")

In [ ]:
# Residual plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Predicted vs Actual
axes[0, 0].scatter(y_val_true, y_val_pred, alpha=0.3, s=10)
axes[0, 0].plot([y_val_true.min(), y_val_true.max()], 
                [y_val_true.min(), y_val_true.max()], 
                'r--', linewidth=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('True Price', fontsize=11)
axes[0, 0].set_ylabel('Predicted Price', fontsize=11)
axes[0, 0].set_title('Predicted vs Actual Prices', fontsize=12, fontweight='bold')
axes[0, 0].legend()

# 2. Residuals vs Predicted
axes[0, 1].scatter(y_val_pred, residuals, alpha=0.3, s=10)
axes[0, 1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Predicted Price', fontsize=11)
axes[0, 1].set_ylabel('Residual', fontsize=11)
axes[0, 1].set_title('Residuals vs Predicted', fontsize=12, fontweight='bold')

# 3. Residual distribution
axes[1, 0].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Residual', fontsize=11)
axes[1, 0].set_ylabel('Frequency', fontsize=11)
axes[1, 0].set_title('Residual Distribution', fontsize=12, fontweight='bold')

# 4. Absolute Error by Price Range
price_bins = pd.qcut(y_val_true, q=10, duplicates='drop')
error_by_price = error_df.groupby(price_bins)['abs_residual'].median().sort_index()
axes[1, 1].plot(range(len(error_by_price)), error_by_price.values, marker='o', linewidth=2)
axes[1, 1].set_xlabel('Price Decile', fontsize=11)
axes[1, 1].set_ylabel('Median Absolute Error', fontsize=11)
axes[1, 1].set_title('Error Increases with Price', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 RESIDUAL PLOT INSIGHTS:")
print("   - Top-left: Points should cluster around red line (perfect prediction)")
print("   - Top-right: Residuals should be randomly scattered around 0 (no pattern)")
print("   - Bottom-left: Residuals should be normally distributed around 0")
print("   - Bottom-right: Error magnitude increases with price (heteroscedasticity)")

### 5.6 Compute Confidence Intervals

In [ ]:
# Compute CI statistics from validation residuals
ci_stats = compute_confidence_intervals(
    model=model,
    X_val=X_val,
    y_val=y_val,
    n_quantiles=3,
    quantile_levels=(0.05, 0.95)
)

---
## 6. Model Interpretability with SHAP

SHAP (SHapley Additive exPlanations) shows:
- Which features drive predictions
- How each feature impacts specific predictions
- Business insights for stakeholders

In [ ]:
import shap

print("Computing SHAP values... (this may take 2-3 minutes)")

# Use a sample of validation data for SHAP (full dataset too slow)
shap_sample_size = min(1000, len(X_val))
X_val_sample = X_val.sample(n=shap_sample_size, random_state=RANDOM_SEED)

# Create SHAP explainer for tree-based models
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_val_sample)

print(f"✓ SHAP values computed for {shap_sample_size} samples")

### 6.1 Global Feature Importance (SHAP Summary Plot)

In [ ]:
# Summary plot - shows feature importance AND direction of impact
print("=" * 80)
print("SHAP SUMMARY PLOT - Global Feature Importance")
print("=" * 80)
print("\nHow to read:")
print("  - Y-axis: Features ranked by importance (top = most important)")
print("  - X-axis: SHAP value (impact on prediction in log-space)")
print("  - Color: Feature value (red = high, blue = low)")
print("  - Each dot: One property")
print("\nExample interpretation:")
print("  If ACTUAL_AREA is top feature with red dots on right:")
print("  → Larger areas (red) push predictions higher (positive SHAP)")
print("\n")

plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_val_sample, max_display=20, show=False)
plt.tight_layout()
plt.show()

In [ ]:
# Bar plot - simple feature importance
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_val_sample, plot_type="bar", max_display=15, show=False)
plt.title('Top 15 Features by Mean Absolute SHAP Value', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### 6.2 Local Explanations - Individual Predictions

Let's explain 3 specific predictions to show how the model reasons.

In [ ]:
# Select 3 representative examples: low, medium, high price
y_val_true_sample = np.expm1(y_val.loc[X_val_sample.index])
low_idx = y_val_true_sample.idxmin()
high_idx = y_val_true_sample.idxmax()
mid_idx = y_val_true_sample.iloc[len(y_val_true_sample)//2]

example_indices = [low_idx, y_val_true_sample.index[len(y_val_true_sample)//2], high_idx]
example_labels = ['Low-Price Property', 'Mid-Price Property', 'High-Price Property']

print("=" * 80)
print("LOCAL SHAP EXPLANATIONS - Individual Property Predictions")
print("=" * 80)

In [ ]:
# Example 1
idx = example_indices[0]
sample_idx = X_val_sample.index.get_loc(idx)

true_price = np.expm1(y_val.loc[idx])
pred_price_log = model.predict(X_val.loc[[idx]])[0]
pred_price = np.expm1(pred_price_log)

print(f"\n{example_labels[0]}:")
print(f"  True Price: {true_price:,.0f}")
print(f"  Predicted Price: {pred_price:,.0f}")
print(f"  Error: {abs(true_price - pred_price):,.0f} ({abs(true_price - pred_price)/true_price*100:.1f}%)")
print(f"\n  Top features driving this prediction:")

# Get SHAP values for this example
shap_vals_example = shap_values[sample_idx]
feature_names = X_val_sample.columns
feature_values = X_val_sample.iloc[sample_idx]

# Sort by absolute SHAP value
shap_importance = pd.DataFrame({
    'feature': feature_names,
    'value': feature_values,
    'shap': shap_vals_example
}).sort_values('shap', key=abs, ascending=False).head(5)

for _, row in shap_importance.iterrows():
    direction = "increases" if row['shap'] > 0 else "decreases"
    print(f"    - {row['feature']} = {row['value']} → {direction} price (SHAP: {row['shap']:+.3f})")

# Waterfall plot
print("\n  SHAP Waterfall Plot:")
shap.waterfall_plot(shap.Explanation(
    values=shap_vals_example,
    base_values=explainer.expected_value,
    data=feature_values,
    feature_names=feature_names.tolist()
), max_display=10, show=False)
plt.tight_layout()
plt.show()

In [ ]:
# Example 2
idx = example_indices[1]
sample_idx = X_val_sample.index.get_loc(idx)

true_price = np.expm1(y_val.loc[idx])
pred_price_log = model.predict(X_val.loc[[idx]])[0]
pred_price = np.expm1(pred_price_log)

print(f"\n{example_labels[1]}:")
print(f"  True Price: {true_price:,.0f}")
print(f"  Predicted Price: {pred_price:,.0f}")
print(f"  Error: {abs(true_price - pred_price):,.0f} ({abs(true_price - pred_price)/true_price*100:.1f}%)")
print(f"\n  Top features driving this prediction:")

shap_vals_example = shap_values[sample_idx]
feature_values = X_val_sample.iloc[sample_idx]

shap_importance = pd.DataFrame({
    'feature': feature_names,
    'value': feature_values,
    'shap': shap_vals_example
}).sort_values('shap', key=abs, ascending=False).head(5)

for _, row in shap_importance.iterrows():
    direction = "increases" if row['shap'] > 0 else "decreases"
    print(f"    - {row['feature']} = {row['value']} → {direction} price (SHAP: {row['shap']:+.3f})")

print("\n  SHAP Waterfall Plot:")
shap.waterfall_plot(shap.Explanation(
    values=shap_vals_example,
    base_values=explainer.expected_value,
    data=feature_values,
    feature_names=feature_names.tolist()
), max_display=10, show=False)
plt.tight_layout()
plt.show()

In [ ]:
# Example 3
idx = example_indices[2]
sample_idx = X_val_sample.index.get_loc(idx)

true_price = np.expm1(y_val.loc[idx])
pred_price_log = model.predict(X_val.loc[[idx]])[0]
pred_price = np.expm1(pred_price_log)

print(f"\n{example_labels[2]}:")
print(f"  True Price: {true_price:,.0f}")
print(f"  Predicted Price: {pred_price:,.0f}")
print(f"  Error: {abs(true_price - pred_price):,.0f} ({abs(true_price - pred_price)/true_price*100:.1f}%)")
print(f"\n  Top features driving this prediction:")

shap_vals_example = shap_values[sample_idx]
feature_values = X_val_sample.iloc[sample_idx]

shap_importance = pd.DataFrame({
    'feature': feature_names,
    'value': feature_values,
    'shap': shap_vals_example
}).sort_values('shap', key=abs, ascending=False).head(5)

for _, row in shap_importance.iterrows():
    direction = "increases" if row['shap'] > 0 else "decreases"
    print(f"    - {row['feature']} = {row['value']} → {direction} price (SHAP: {row['shap']:+.3f})")

print("\n  SHAP Waterfall Plot:")
shap.waterfall_plot(shap.Explanation(
    values=shap_vals_example,
    base_values=explainer.expected_value,
    data=feature_values,
    feature_names=feature_names.tolist()
), max_display=10, show=False)
plt.tight_layout()
plt.show()

### 6.3 Business Insights from SHAP Analysis

In [ ]:
print("=" * 80)
print("KEY BUSINESS INSIGHTS FROM MODEL")
print("=" * 80)

# Get mean absolute SHAP values across all samples
mean_abs_shap = np.abs(shap_values).mean(axis=0)
shap_importance_global = pd.DataFrame({
    'feature': X_val_sample.columns,
    'importance': mean_abs_shap
}).sort_values('importance', ascending=False)

top_driver = shap_importance_global.iloc[0]['feature']

print(f"\n1. PRIMARY PRICE DRIVER: {top_driver}")
if 'ACTUAL_AREA' in top_driver:
    print("   → Property size is the #1 determinant of price")
    print("   → Recommendation: Emphasize sqm/sqft in property listings")
elif 'median_price' in top_driver or 'AREA_EN' in top_driver:
    print("   → Location premium is the #1 determinant of price")
    print("   → Recommendation: Invest in premium areas for higher returns")

print("\n2. LOCATION EFFECTS:")
location_features = [f for f in shap_importance_global['feature'].head(10) 
                     if 'AREA' in f or 'PROJECT' in f or 'median_price' in f]
if location_features:
    print(f"   → {len(location_features)} location-related features in top 10")
    print("   → Location is critical for accurate valuation")

print("\n3. TEMPORAL TRENDS:")
time_features = [f for f in shap_importance_global['feature'].head(20) 
                 if any(x in f.lower() for x in ['year', 'month', 'quarter', 'days'])]
if time_features:
    print(f"   → Time-based features: {time_features[:3]}")
    print("   → Market timing matters for pricing")

print("\n4. PROPERTY CHARACTERISTICS:")
print("   → Size (area), rooms, parking all influence price")
print("   → Larger, well-equipped properties command premium")

print("\n5. MODEL RECOMMENDATIONS FOR PRODUCTION:")
print("   ✓ Model is explainable - can justify predictions to users")
print("   ✓ Location aggregates add significant predictive power")
print("   ✓ Handles unseen properties via fallback to area/type medians")
print("   ✓ Confidence intervals provide uncertainty estimates")

print("\n" + "=" * 80)

---
## 7. Save Model Artifact for Production

Save everything needed for API deployment in a single artifact file.

In [ ]:
# Save complete model artifact
save_model_artifact(
    model=model,
    preprocessing_metadata=preprocessing_metadata,
    ci_stats=ci_stats,
    feature_importance=feature_importance,
    metrics=metrics,
    save_path='../models/trained_model.pkl',
    area_unit=AREA_UNIT
)

---
## 8. Final Summary

### Model Performance
- **Validation R²:** {metrics['val']['r2']:.4f}
- **Validation MAE:** {metrics['val']['mae']:,.0f}
- **Test R²:** {metrics['test']['r2']:.4f}
- **Test MAE:** {metrics['test']['mae']:,.0f}

### Key Achievements
✓ Complete EDA with business insights  
✓ Leak-safe feature engineering  
✓ CatBoost model with MAE loss  
✓ Comprehensive evaluation (overall + segmented)  
✓ SHAP interpretability with business insights  
✓ Confidence intervals for predictions  
✓ Production-ready model artifact  

### Next Steps
1. Deploy FastAPI service using saved artifact
2. Write REPORT.md with technical documentation
3. Write README.md with setup instructions
4. Test end-to-end pipeline

---
**Analysis Complete!** ✅